<a href="https://githubtocolab.com/giswqs/geemap/blob/master/examples/notebooks/usda_naip_imagery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [ ]:
# !pip install geemap

In [1]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

In [ ]:
import ee

In [2]:
import geemap
from ipyleaflet import *
from ipywidgets import *
from geemap.utils import *

Map = geemap.Map(center=(-100, 40), zoom=4)
Map.default_style = {'cursor': 'pointer'}
Map

Map.setOptions('ROADMAP')
# Load National Hydrography Dataset (NHD) 
HUC10 = ee.FeatureCollection('USGS/WBD/2017/HUC10');  # 18,487 HUC10 watersheds in the U.S.

# Add HUC layer to the map
Map.setCenter(-99.00, 47.01, 8);
Map.addLayer(ee.Image().paint(HUC10, 0, 1), {}, 'HUC-10 Watershed')   # HUC10 for the entire U.S.

label = Label("Click on the map to select a watershed")
widget_control = WidgetControl(widget=label, position='bottomright')
Map.add_control(widget_control)

layer = None

def handle_interaction(**kwargs):
    latlon = kwargs.get('coordinates')
    if kwargs.get('type') == 'click':
        Map.default_style = {'cursor': 'wait'}
        xy = ee.Geometry.Point(latlon[::-1])
        watershed = HUC10.filterBounds(xy)
        huc10_id = watershed.first().get('huc10').getInfo()
        Map.layers = Map.layers[:3]
        Map.addLayer(ee.Image().paint(watershed, 0, 2), {'palette': 'red'}, 'HUC ID: ' + huc10_id) 
        
        NAIP_images = find_NAIP(watershed)
        first_image = ee.Image(NAIP_images.toList(5).get(0))
        Map.addLayer(first_image, {'bands': ['N', 'R', 'G']}, 'first image')
        count = NAIP_images.size().getInfo()
        for i in range(0, count):
            image = ee.Image(NAIP_images.toList(count).get(i))
            Map.addLayer(image, {'bands': ['N', 'R', 'G']}, str(i))
        
        Map.default_style = {'cursor': 'pointer'}

Map.on_interaction(handle_interaction)

Map

Map(center=[47.01, -99.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

In [ ]:
Map.layers